# অ্যাটেনশন মেকানিজম এবং ট্রান্সফর্মার

রেকারেন্ট নেটওয়ার্কগুলোর একটি বড় সীমাবদ্ধতা হলো, একটি সিকোয়েন্সের সব শব্দই ফলাফলের উপর সমান প্রভাব ফেলে। এর ফলে সিকোয়েন্স-টু-সিকোয়েন্স কাজের জন্য স্ট্যান্ডার্ড LSTM এনকোডার-ডিকোডার মডেলগুলোতে অপ্টিমাল পারফরম্যান্স হয় না, যেমন নেমড এন্টিটি রিকগনিশন এবং মেশিন ট্রান্সলেশন। বাস্তবে, ইনপুট সিকোয়েন্সের নির্দিষ্ট কিছু শব্দ সিকোয়েন্সিয়াল আউটপুটে অন্য শব্দগুলোর চেয়ে বেশি প্রভাব ফেলে।

ধরা যাক একটি সিকোয়েন্স-টু-সিকোয়েন্স মডেল, যেমন মেশিন ট্রান্সলেশন। এটি দুটি রেকারেন্ট নেটওয়ার্ক দ্বারা বাস্তবায়িত হয়, যেখানে একটি নেটওয়ার্ক (**এনকোডার**) ইনপুট সিকোয়েন্সকে হিডেন স্টেটে রূপান্তর করে এবং অন্যটি, **ডিকোডার**, এই হিডেন স্টেটকে অনুবাদিত ফলাফলে রূপান্তর করে। এই পদ্ধতির সমস্যা হলো, নেটওয়ার্কের চূড়ান্ত স্টেটের জন্য বাক্যের শুরুটা মনে রাখা কঠিন হয়ে যায়, যার ফলে দীর্ঘ বাক্যের ক্ষেত্রে মডেলের গুণগত মান খারাপ হয়।

**অ্যাটেনশন মেকানিজম** একটি পদ্ধতি প্রদান করে, যা প্রতিটি ইনপুট ভেক্টরের প্রাসঙ্গিক প্রভাবকে RNN-এর প্রতিটি আউটপুট প্রেডিকশনে ওজন দেয়। এটি বাস্তবায়িত হয় ইনপুট RNN-এর মধ্যবর্তী স্টেট এবং আউটপুট RNN-এর মধ্যে শর্টকাট তৈরি করে। এই পদ্ধতিতে, যখন আউটপুট প্রতীক $y_t$ তৈরি করা হয়, তখন আমরা সব ইনপুট হিডেন স্টেট $h_i$-কে বিভিন্ন ওজন সহগ $\alpha_{t,i}$ সহ বিবেচনা করব।

![এনকোডার/ডিকোডার মডেল একটি অ্যাডিটিভ অ্যাটেনশন লেয়ার সহ](../../../../../translated_images/bn/encoder-decoder-attention.7a726296894fb567.webp)
*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) থেকে উদ্ধৃত, [এই ব্লগ পোস্ট](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html) থেকে নেওয়া এনকোডার-ডিকোডার মডেল অ্যাডিটিভ অ্যাটেনশন মেকানিজম সহ]*

অ্যাটেনশন ম্যাট্রিক্স $\{\alpha_{i,j}\}$ নির্দেশ করে যে নির্দিষ্ট ইনপুট শব্দগুলো আউটপুট সিকোয়েন্সের একটি নির্দিষ্ট শব্দ তৈরিতে কতটা ভূমিকা রাখে। নিচে এমন একটি ম্যাট্রিক্সের উদাহরণ দেওয়া হলো:

![RNNsearch-50 দ্বারা পাওয়া একটি নমুনা অ্যালাইনমেন্ট, Bahdanau - arviz.org থেকে নেওয়া](../../../../../translated_images/bn/bahdanau-fig3.09ba2d37f202a6af.webp)

*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) থেকে নেওয়া (Fig.3)*

অ্যাটেনশন মেকানিজম বর্তমান বা প্রায় বর্তমান প্রাকৃতিক ভাষা প্রক্রিয়াকরণের (NLP) স্টেট অফ দ্য আর্টের জন্য দায়ী। তবে অ্যাটেনশন যোগ করার ফলে মডেলের প্যারামিটারের সংখ্যা উল্লেখযোগ্যভাবে বৃদ্ধি পায়, যা RNN-এর স্কেলিং সমস্যার দিকে নিয়ে যায়। RNN-এর স্কেলিংয়ের একটি প্রধান সীমাবদ্ধতা হলো, মডেলের রেকারেন্ট প্রকৃতি প্রশিক্ষণকে ব্যাচ এবং প্যারালালাইজ করা কঠিন করে তোলে। RNN-এ একটি সিকোয়েন্সের প্রতিটি উপাদান ক্রমানুসারে প্রক্রিয়া করতে হয়, যার ফলে এটি সহজে প্যারালালাইজ করা যায় না।

অ্যাটেনশন মেকানিজমের গ্রহণযোগ্যতা এবং এই সীমাবদ্ধতার সংমিশ্রণে বর্তমান স্টেট অফ দ্য আর্ট ট্রান্সফর্মার মডেলের সৃষ্টি হয়েছে, যা আমরা আজ BERT থেকে OpenGPT3 পর্যন্ত ব্যবহার করি।

## ট্রান্সফর্মার মডেল

প্রতিটি পূর্ববর্তী প্রেডিকশনের প্রসঙ্গকে পরবর্তী মূল্যায়ন ধাপে ফরোয়ার্ড করার পরিবর্তে, **ট্রান্সফর্মার মডেল** **পজিশনাল এনকোডিং** এবং অ্যাটেনশন ব্যবহার করে একটি প্রদত্ত টেক্সট উইন্ডোর মধ্যে একটি নির্দিষ্ট ইনপুটের প্রসঙ্গ ধারণ করে। নিচের ছবিটি দেখায় কিভাবে পজিশনাল এনকোডিং এবং অ্যাটেনশন একটি প্রদত্ত উইন্ডোর মধ্যে প্রসঙ্গ ধারণ করতে পারে।

![ট্রান্সফর্মার মডেলে মূল্যায়ন কীভাবে সম্পন্ন হয় তা দেখানো একটি অ্যানিমেটেড GIF।](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

যেহেতু প্রতিটি ইনপুট পজিশন স্বাধীনভাবে প্রতিটি আউটপুট পজিশনে ম্যাপ করা হয়, ট্রান্সফর্মার RNN-এর চেয়ে ভালোভাবে প্যারালালাইজ করতে পারে, যা অনেক বড় এবং আরও এক্সপ্রেসিভ ভাষার মডেল সক্ষম করে। প্রতিটি অ্যাটেনশন হেড বিভিন্ন শব্দের মধ্যে সম্পর্ক শিখতে ব্যবহার করা যেতে পারে, যা ডাউনস্ট্রিম NLP কাজগুলো উন্নত করে।

**BERT** (Bidirectional Encoder Representations from Transformers) একটি খুব বড় মাল্টি-লেয়ার ট্রান্সফর্মার নেটওয়ার্ক, যেখানে *BERT-base*-এর জন্য ১২টি লেয়ার এবং *BERT-large*-এর জন্য ২৪টি লেয়ার রয়েছে। মডেলটি প্রথমে একটি বড় টেক্সট ডেটাসেট (উইকিপিডিয়া + বই) ব্যবহার করে আনসুপারভাইজড প্রশিক্ষণের মাধ্যমে প্রি-ট্রেইন করা হয় (একটি বাক্যে মাস্ক করা শব্দগুলো প্রেডিক্ট করা)। প্রি-ট্রেইনিংয়ের সময় মডেলটি উল্লেখযোগ্য ভাষাগত বোঝাপড়া অর্জন করে, যা পরে অন্যান্য ডেটাসেটের সাথে ফাইন টিউনিংয়ের মাধ্যমে ব্যবহার করা যায়। এই প্রক্রিয়াটিকে **ট্রান্সফার লার্নিং** বলা হয়।

![http://jalammar.github.io/illustrated-bert/ থেকে নেওয়া ছবি](../../../../../translated_images/bn/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

ট্রান্সফর্মার আর্কিটেকচারের অনেক ভেরিয়েশন রয়েছে, যেমন BERT, DistilBERT, BigBird, OpenGPT3 এবং আরও অনেক, যেগুলো ফাইন টিউন করা যায়। [HuggingFace প্যাকেজ](https://github.com/huggingface/) PyTorch ব্যবহার করে এই আর্কিটেকচারগুলোর অনেকগুলোর প্রশিক্ষণের জন্য একটি রিপোজিটরি প্রদান করে।

## টেক্সট ক্লাসিফিকেশনের জন্য BERT ব্যবহার

চলুন দেখি কিভাবে আমরা প্রি-ট্রেইনড BERT মডেল ব্যবহার করে আমাদের ঐতিহ্যবাহী কাজ সমাধান করতে পারি: সিকোয়েন্স ক্লাসিফিকেশন। আমরা আমাদের মূল AG News ডেটাসেটটি ক্লাসিফাই করব।

প্রথমে, চলুন HuggingFace লাইব্রেরি এবং আমাদের ডেটাসেট লোড করি:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


আমরা যেহেতু প্রি-ট্রেইনড BERT মডেল ব্যবহার করব, তাই আমাদের নির্দিষ্ট একটি টোকেনাইজার ব্যবহার করতে হবে। প্রথমে, আমরা প্রি-ট্রেইনড BERT মডেলের সাথে সম্পর্কিত একটি টোকেনাইজার লোড করব।

HuggingFace লাইব্রেরিতে প্রি-ট্রেইনড মডেলের একটি সংগ্রহশালা রয়েছে, যা আপনি শুধুমাত্র তাদের নাম `from_pretrained` ফাংশনের আর্গুমেন্ট হিসেবে উল্লেখ করেই ব্যবহার করতে পারেন। মডেলের জন্য প্রয়োজনীয় সমস্ত বাইনারি ফাইল স্বয়ংক্রিয়ভাবে ডাউনলোড হয়ে যাবে।

তবে, কিছু ক্ষেত্রে আপনাকে নিজের মডেল লোড করতে হতে পারে। এই পরিস্থিতিতে, আপনি সেই ডিরেক্টরির অবস্থান উল্লেখ করতে পারেন যেখানে সমস্ত প্রাসঙ্গিক ফাইল রয়েছে, যেমন টোকেনাইজারের জন্য প্যারামিটার, মডেলের প্যারামিটার সহ `config.json` ফাইল, বাইনারি ওজন ইত্যাদি।


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` অবজেক্টে `encode` ফাংশন রয়েছে যা সরাসরি টেক্সট এনকোড করতে ব্যবহার করা যেতে পারে:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

তারপর, চলুন ট্রেনিংয়ের সময় ডেটা অ্যাক্সেস করার জন্য আমরা যে ইটারেটরগুলি ব্যবহার করব তা তৈরি করি। কারণ BERT তার নিজস্ব এনকোডিং ফাংশন ব্যবহার করে, আমাদের একটি প্যাডিং ফাংশন সংজ্ঞায়িত করতে হবে যা পূর্বে সংজ্ঞায়িত `padify` এর অনুরূপ:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

আমাদের ক্ষেত্রে, আমরা `bert-base-uncased` নামে একটি প্রি-ট্রেইনড BERT মডেল ব্যবহার করব। চলুন `BertForSequenceClassfication` প্যাকেজ ব্যবহার করে মডেলটি লোড করি। এটি নিশ্চিত করে যে আমাদের মডেলের ক্লাসিফিকেশনের জন্য প্রয়োজনীয় আর্কিটেকচার ইতিমধ্যেই রয়েছে, যার মধ্যে ফাইনাল ক্লাসিফায়ারও অন্তর্ভুক্ত। আপনি একটি সতর্কবার্তা দেখতে পাবেন যেখানে বলা হবে যে ফাইনাল ক্লাসিফায়ারের ওজনগুলি ইনিশিয়ালাইজ করা হয়নি, এবং মডেলটি প্রি-ট্রেইনিং প্রয়োজন - এটি সম্পূর্ণ ঠিক আছে, কারণ আমরা ঠিক সেটাই করতে যাচ্ছি!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

এখন আমরা প্রশিক্ষণ শুরু করতে প্রস্তুত! যেহেতু BERT ইতিমধ্যেই প্রি-ট্রেইন করা আছে, আমরা খুব ছোট লার্নিং রেট দিয়ে শুরু করতে চাই যাতে প্রাথমিক ওজন নষ্ট না হয়।

সমস্ত কঠিন কাজ `BertForSequenceClassification` মডেল দ্বারা সম্পন্ন হয়। যখন আমরা প্রশিক্ষণের ডেটায় মডেলটি কল করি, এটি ইনপুট মিনিব্যাচের জন্য ক্ষতি এবং নেটওয়ার্ক আউটপুট উভয়ই প্রদান করে। আমরা প্যারামিটার অপ্টিমাইজেশনের জন্য ক্ষতি ব্যবহার করি (`loss.backward()` ব্যাকওয়ার্ড পাস সম্পন্ন করে), এবং প্রশিক্ষণের সঠিকতা গণনার জন্য `out` ব্যবহার করি, যেখানে প্রাপ্ত লেবেল `labs` (যা `argmax` ব্যবহার করে গণনা করা হয়) প্রত্যাশিত `labels` এর সাথে তুলনা করা হয়।

প্রক্রিয়াটি নিয়ন্ত্রণ করার জন্য, আমরা কয়েকটি পুনরাবৃত্তির উপর ক্ষতি এবং সঠিকতা জমা করি এবং প্রতি `report_freq` প্রশিক্ষণ চক্রে সেগুলি প্রিন্ট করি।

এই প্রশিক্ষণ সম্ভবত বেশ দীর্ঘ সময় নিতে পারে, তাই আমরা পুনরাবৃত্তির সংখ্যা সীমিত করি।


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


আপনি দেখতে পাবেন (বিশেষত যদি আপনি পুনরাবৃত্তির সংখ্যা বাড়ান এবং যথেষ্ট সময় অপেক্ষা করেন) যে BERT শ্রেণীবিভাগ আমাদের বেশ ভালো নির্ভুলতা প্রদান করে! এর কারণ হলো BERT ইতিমধ্যেই ভাষার গঠন বেশ ভালোভাবে বোঝে, এবং আমাদের কেবলমাত্র চূড়ান্ত শ্রেণীবিভাজকটি সূক্ষ্মভাবে সামঞ্জস্য করতে হয়। তবে, যেহেতু BERT একটি বড় মডেল, পুরো প্রশিক্ষণ প্রক্রিয়াটি অনেক সময় নেয় এবং উল্লেখযোগ্য কম্পিউটেশনাল ক্ষমতার প্রয়োজন হয়! (GPU, এবং সম্ভব হলে একাধিক GPU)।

> **Note:** আমাদের উদাহরণে, আমরা সবচেয়ে ছোট প্রি-ট্রেইনড BERT মডেলগুলোর একটি ব্যবহার করেছি। বড় মডেলগুলো সম্ভবত আরও ভালো ফলাফল প্রদান করতে পারে।


## মডেলের কার্যকারিতা মূল্যায়ন

এখন আমরা টেস্ট ডেটাসেটে আমাদের মডেলের কার্যকারিতা মূল্যায়ন করতে পারি। মূল্যায়ন প্রক্রিয়া প্রশিক্ষণ প্রক্রিয়ার সাথে বেশ মিল, তবে আমরা যেন ভুলে না যাই যে `model.eval()` কল করে মডেলকে মূল্যায়ন মোডে স্যুইচ করতে হবে।


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## মূল বিষয়বস্তু

এই ইউনিটে, আমরা দেখেছি কীভাবে **transformers** লাইব্রেরি থেকে প্রি-ট্রেইনড ভাষার মডেল সহজেই নিয়ে আমাদের টেক্সট ক্লাসিফিকেশন টাস্কে ব্যবহার করা যায়। একইভাবে, BERT মডেলগুলো ব্যবহার করা যেতে পারে এন্টিটি এক্সট্রাকশন, প্রশ্নোত্তর এবং অন্যান্য NLP কাজের জন্য।

ট্রান্সফরমার মডেলগুলো NLP-তে বর্তমান সময়ের সর্বাধুনিক প্রযুক্তি উপস্থাপন করে, এবং বেশিরভাগ ক্ষেত্রেই এটি হওয়া উচিত প্রথম সমাধান যার মাধ্যমে আপনি কাস্টম NLP সমাধান বাস্তবায়নের সময় পরীক্ষা-নিরীক্ষা শুরু করবেন। তবে, এই মডিউলে আলোচনা করা পুনরাবৃত্ত নিউরাল নেটওয়ার্কের মৌলিক নীতিগুলো বোঝা অত্যন্ত গুরুত্বপূর্ণ যদি আপনি উন্নত নিউরাল মডেল তৈরি করতে চান।



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়ী থাকব না।
